### MPDA - Flappy Bird Reinforcement Learning


### PROGRESS:
[Date] [Name]: [What was done]

[30.11.] [mszuc]: [Team work templates, Define tasks for project]

[1.12.] [xsocha02]: [Added random baseline agent + video/GIF]


### BLOCKERS:
None yet

### DECISIONS:
- Using flappy-bird-gymnasium package for environment
- Starting with simple feature space (12 values)
- PyTorch for neural networks

In [ ]:
# ═══════════════════════════════════════════════════════════
# SECTION 0: SETUP & INSTALLATION
# ═══════════════════════════════════════════════════════════

!pip install flappy-bird-gymnasium torch torchvision matplotlib imageio

import gymnasium as gym
import flappy_bird_gymnasium
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import random
from gymnasium.wrappers import RecordVideo
import imageio
# Note: moviepy removed - using imageio for GIF creation instead


/Users/mszuc/school/fekt-pda-project/venv/lib/python3.11/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [2]:
# Test environment loads
env = gym.make('FlappyBird-v0', render_mode='rgb_array', use_lidar=True)
print(f"✅ Environment loaded!")
print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")

✅ Environment loaded!
Observation space: Box(0.0, 1.0, (180,), float64)
Action space: Discrete(2)


In [3]:
# Render test
env.reset()
env.render()

/Users/mszuc/school/fekt-pda-project/venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


array([[[ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202],
        ...,
        [ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202]],

       [[ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202],
        ...,
        [ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202]],

       [[ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202],
        ...,
        [ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202]],

       ...,

       [[222, 216, 149],
        [222, 216, 149],
        [222, 216, 149],
        ...,
        [222, 216, 149],
        [222, 216, 149],
        [222, 216, 149]],

       [[222, 216, 149],
        [222, 216, 149],
        [222, 216, 149],
        ...,
        [222, 216, 149],
        [222, 216, 149],
        [222, 216, 149]],

       [[222, 216, 149],
        [222, 216, 149],
        [222, 216, 149],
        ...,
        [222, 216, 149],
        [222, 216, 149],
        [222, 216, 149]]

In [4]:
action = 0
env.step(action)

env.render()

/Users/mszuc/school/fekt-pda-project/venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


array([[[ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202],
        ...,
        [ 78, 192, 202],
        [ 84,  56,  71],
        [ 84,  56,  71]],

       [[ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202],
        ...,
        [ 78, 192, 202],
        [ 84,  56,  71],
        [ 84,  56,  71]],

       [[ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202],
        ...,
        [ 78, 192, 202],
        [ 84,  56,  71],
        [ 84,  56,  71]],

       ...,

       [[222, 216, 149],
        [222, 216, 149],
        [222, 216, 149],
        ...,
        [222, 216, 149],
        [222, 216, 149],
        [222, 216, 149]],

       [[222, 216, 149],
        [222, 216, 149],
        [222, 216, 149],
        ...,
        [222, 216, 149],
        [222, 216, 149],
        [222, 216, 149]],

       [[222, 216, 149],
        [222, 216, 149],
        [222, 216, 149],
        ...,
        [222, 216, 149],
        [222, 216, 149],
        [222, 216, 149]]

In [5]:
action = 1
env.step(action)

env.render()

array([[[ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202],
        ...,
        [132, 169,  68],
        [148, 183,  81],
        [148, 183,  81]],

       [[ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202],
        ...,
        [132, 169,  68],
        [148, 183,  81],
        [148, 183,  81]],

       [[ 78, 192, 202],
        [ 78, 192, 202],
        [ 78, 192, 202],
        ...,
        [132, 170,  69],
        [148, 183,  81],
        [148, 183,  81]],

       ...,

       [[222, 216, 149],
        [222, 216, 149],
        [222, 216, 149],
        ...,
        [222, 216, 149],
        [222, 216, 149],
        [222, 216, 149]],

       [[222, 216, 149],
        [222, 216, 149],
        [222, 216, 149],
        ...,
        [222, 216, 149],
        [222, 216, 149],
        [222, 216, 149]],

       [[222, 216, 149],
        [222, 216, 149],
        [222, 216, 149],
        ...,
        [222, 216, 149],
        [222, 216, 149],
        [222, 216, 149]]

In [6]:
env.close()

In [ ]:
# ═══════════════════════════════════════════════════════════
# SECTION 1: HYPERPARAMETERS
# ═══════════════════════════════════════════════════════════

# FIXED: Lower learning rate for stability (was 1e-3)
LEARNING_RATE = 5e-4
GAMMA = 0.99
EPSILON_START = 1.0
EPSILON_END = 0.01
# FIXED: Epsilon decay is now applied PER EPISODE, not per step
# With 0.995, after 500 episodes: 0.995^500 ≈ 0.08 (good exploration)
EPSILON_DECAY = 0.995
BATCH_SIZE = 64
MEMORY_SIZE = 10000
# FIXED: More frequent target updates for stability (was 100)
TARGET_UPDATE = 50
NUM_EPISODES = 1000  # Reduced for faster iteration, increase for final training
MODEL_PATH = 'flappy_bird_dqn_v1.pth' # Must match the save path


In [8]:
# ═══════════════════════════════════════════════════════════
# SECTION 2: ENVIRONMENT EXPLORATION
# ═══════════════════════════════════════════════════════════

def explore_environment():
    """
    Understand state/action space of Flappy Bird.
    Prints shapes, runs random episode.
    """
    # TODO: Implement exploration
    observation, info = env.reset()
    print(f"Observation shape: {observation.shape}")
    print(f"Info: {info}")

    print(f"Last 10 values: {observation[-10:]}")

    # jump
    action = 1
    observation, reward, terminated, truncated, info = env.step(action)

    # bird jumped up => lower values increased
    print(f"After action: {observation[-10:]}")

    # don't jump
    action = 0
    observation, reward, terminated, truncated, info = env.step(action)

    # still rising
    print(f"After action: {observation[-10:]}")
    action = 0

env = gym.make('FlappyBird-v0', render_mode='rgb_array', use_lidar=True)
explore_environment()
env.close()

Observation shape: (180,)
Info: {'score': 0}
Last 10 values: [0.87077653 0.86325566 0.85355176 0.84422975 0.83749654 0.83099074
 0.82471771 0.81868282 0.81289137 0.80734859]
After action: [0.92584822 0.91578435 0.90606844 0.8967118  0.88772578 0.88123633
 0.87496653 0.86892114 0.86310487 0.85752239]
After action: [0.97139963 0.96137938 0.95169132 0.9423457  0.93335279 0.92684578
 0.91849489 0.91248114 0.90668591 0.90111343]


In [ ]:
# ═══════════════════════════════════════════════════════════
# SECTION 3: RANDOM BASELINE AGENT
# ═══════════════════════════════════════════════════════════

def random_agent_baseline(env, num_episodes=10):
    """
    Random agent for baseline performance.

    Returns:
        avg_score: Average score over episodes
        avg_steps: Average survival time
    """
    scores = []
    steps = []
    for episode in range(num_episodes):
        observation, info = env.reset()
        done = False
        score = 0
        steps_taken = 0
        while not done:
            action = env.action_space.sample()
            observation, reward, terminated, truncated, info = env.step(action)
            score += reward
            done = terminated or truncated
            steps_taken += 1
        scores.append(score)
        steps.append(steps_taken)

    avg_score = np.mean(scores)
    avg_steps = np.mean(steps)

    return avg_score, avg_steps

env = gym.make('FlappyBird-v0', render_mode='rgb_array', use_lidar=True)
env = RecordVideo(env, 'random_agent_video', episode_trigger=lambda x: True)
avg_score, avg_steps = random_agent_baseline(env)
env.close()

print(f"Average score: {avg_score}")
print(f"Average steps: {avg_steps}")
print("Videos saved in /content/random_agent_video/")

# convert to gif (using imageio instead of moviepy for better compatibility)
import imageio
reader = imageio.get_reader("random_agent_video/rl-video-episode-0.mp4")
frames = [frame for frame in reader]
imageio.mimsave("random_agent.gif", frames, fps=30)
reader.close()
print("GIF saved as random_agent.gif")

# When randomly choosing actions, bird almost always flies up to the top of the screen

Average score: -7.079999999999998
Average steps: 50.0
Videos saved in /content/random_agent_video/


TypeError: got an unexpected keyword argument 'program'

In [ ]:
# ═══════════════════════════════════════════════════════════
# SECTION 4: DQN NETWORK
# ═══════════════════════════════════════════════════════════

class DQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        """
        Deep Q-Network.

        Args:
            state_dim: Dimension of state space (12 for features, 180 for LIDAR)
            action_dim: Number of actions (2: nothing/flap)
        """
        super(DQN, self).__init__()

        # 2–3 fully connected layers with ReLU
        hidden1 = 128
        hidden2 = 128

        self.net = nn.Sequential(
            nn.Linear(state_dim, hidden1),
            nn.ReLU(),
            nn.Linear(hidden1, hidden2),
            nn.ReLU(),
            nn.Linear(hidden2, action_dim)
        )

    def forward(self, x):
        """
        Forward pass.

        Args:
            x: State tensor [batch_size, state_dim]

        Returns:
            Q-values: [batch_size, action_dim]
        """
        return self.net(x)


# -----------------------------------------------------------
# Model Save / Load
# -----------------------------------------------------------pabad

def save_dqn(model, path):
    """
    Save DQN model weights to file.
    """
    torch.save(model.state_dict(), path)


def load_dqn(path, state_dim, action_dim, device="cpu"):
    """
    Load DQN model weights from file.

    Returns:
        model: DQN instance with loaded weights (in eval mode)
    """
    model = DQN(state_dim, action_dim).to(device)
    state_dict = torch.load(path, map_location=device)
    model.load_state_dict(state_dict)
    model.eval()
    return model


#test
env = gym.make('FlappyBird-v0', use_lidar=True)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

model = DQN(state_dim, action_dim)
test_state = torch.randn(1, state_dim)   # fake input
q_values = model(test_state)

print("State dim:", state_dim)
print("Action dim:", action_dim)
print("Q-values:", q_values)

In [ ]:
# ═══════════════════════════════════════════════════════════
# SECTION 5: EXPERIENCE REPLAY BUFFER
# ═══════════════════════════════════════════════════════════

from collections import deque
import random
import numpy as np

class ReplayBuffer:
    def __init__(self, capacity):
        """
        Experience replay buffer.

        Args:
            capacity: Maximum number of experiences to store
        """
        self.capacity = capacity
        self.buffer = deque(maxlen=capacity)   # auto-removes old items

    def push(self, state, action, reward, next_state, done):
        """Store one experience into the buffer."""
        experience = (state, action, reward, next_state, done)
        self.buffer.append(experience)

    def sample(self, batch_size):
        """Sample a random batch of experiences."""
        batch = random.sample(self.buffer, batch_size)

        # Unzip the batch
        states, actions, rewards, next_states, dones = zip(*batch)

        # Convert to NumPy arrays (PyTorch will convert later)
        return (
            np.array(states),
            np.array(actions),
            np.array(rewards, dtype=np.float32),
            np.array(next_states),
            np.array(dones, dtype=np.float32),
        )

    def __len__(self):
        """Return current size of the buffer."""
        return len(self.buffer)


#test
buf = ReplayBuffer(100)

buf.push([1,2,3], 1, 0.5, [1,2,4], False)
buf.push([4,5,6], 0, 1.0, [4,5,7], True)

print("Buffer size:", len(buf))
batch = buf.sample(2)

print("Sampled states:", batch[0])
print("Sampled actions:", batch[1])

In [ ]:
# ═══════════════════════════════════════════════════════════
# SECTION 6: DQN AGENT
class DQNAgent:
    def __init__(self, state_dim, action_dim, device=None):
        """
        DQN Agent with target network.

        Args:
            state_dim: State space dimension
            action_dim: Action space dimension
        """
        self.state_dim = state_dim
        self.action_dim = action_dim

        # Device (CPU / GPU)
        if device is None:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.device = device

        # Hyperparameters
        self.gamma = 0.99           # discount factor
        self.batch_size = 64
        self.learning_rate = LEARNING_RATE  # Use global hyperparameter
        self.replay_capacity = 50_000
        self.min_buffer_size = 1_000  # start training only after this many transitions

        # Networks
        self.policy_net = DQN(state_dim, action_dim).to(self.device)
        self.target_net = DQN(state_dim, action_dim).to(self.device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()  # target net in eval mode

        # Optimizer & loss
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.learning_rate)
        self.loss_fn = nn.MSELoss()

        # Replay buffer
        self.replay_buffer = ReplayBuffer(self.replay_capacity)

    def select_action(self, state, epsilon):
        """
        Epsilon-greedy action selection.

        Args:
            state: Current state (np.array shape [state_dim])
            epsilon: Exploration probability

        Returns:
            action: 0 or 1
        """
        # Explore
        if random.random() < epsilon:
            return random.randrange(self.action_dim)

        # Exploit
        state_tensor = torch.tensor(
            state, dtype=torch.float32, device=self.device
        ).unsqueeze(0)  # [1, state_dim]

        with torch.no_grad():
            q_values = self.policy_net(state_tensor)  # [1, action_dim]
            action = q_values.argmax(dim=1).item()

        return action

    def train_step(self):
        """
        Sample batch and update network.

        Returns:
            loss_value: float or None (if not enough data)
        """
        # Don't train until buffer has enough samples
        if len(self.replay_buffer) < max(self.batch_size, self.min_buffer_size):
            return None

        # Sample batch from replay buffer
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.batch_size)

        # Convert to tensors
        states      = torch.tensor(states, dtype=torch.float32, device=self.device)
        actions     = torch.tensor(actions, dtype=torch.long,   device=self.device).unsqueeze(1)   # [B,1]
        rewards     = torch.tensor(rewards, dtype=torch.float32, device=self.device).unsqueeze(1) # [B,1]
        next_states = torch.tensor(next_states, dtype=torch.float32, device=self.device)
        dones       = torch.tensor(dones, dtype=torch.float32,  device=self.device).unsqueeze(1)  # [B,1]

        # Current Q-values from policy network for taken actions
        q_values = self.policy_net(states).gather(1, actions)  # [B,1]

        # Target Q-values using target network
        with torch.no_grad():
            next_q_values = self.target_net(next_states).max(dim=1, keepdim=True)[0]  # [B,1]
            target_q_values = rewards + self.gamma * (1.0 - dones) * next_q_values

        # Compute loss
        loss = self.loss_fn(q_values, target_q_values)

        # Backpropagation
        self.optimizer.zero_grad()
        loss.backward()
        # FIXED: Enable gradient clipping for training stability
        torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), 1.0)
        self.optimizer.step()

        return loss.item()

    def update_target_network(self):
        """Copy weights from policy to target network."""
        self.target_net.load_state_dict(self.policy_net.state_dict())

env = gym.make("FlappyBird-v0", use_lidar=True)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

agent = DQNAgent(state_dim, action_dim)

state, _ = env.reset()
action = agent.select_action(state, epsilon=0.5)
print("Sampled action:", action)

env.close()

In [ ]:
# ═══════════════════════════════════════════════════════════
# SECTION 8: HELPER FUNCTIONS
# ═══════════════════════════════════════════════════════════

def moving_average(data, window=50):
    """Calculate moving average for smoother visualization."""
    if len(data) < window:
        return data
    return np.convolve(data, np.ones(window)/window, mode='valid')

def plot_training_results(rewards, losses):
    """Plot reward curves and training losses with moving averages."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Rewards plot
    axes[0].plot(rewards, alpha=0.3, label='Raw rewards')
    if len(rewards) >= 50:
        ma_rewards = moving_average(rewards, 50)
        axes[0].plot(range(49, len(rewards)), ma_rewards, label='Moving avg (50 ep)', linewidth=2)
    axes[0].set_xlabel('Episode')
    axes[0].set_ylabel('Reward')
    axes[0].set_title('Training Rewards')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Loss plot
    axes[1].plot(losses, alpha=0.3, label='Raw loss')
    if len(losses) >= 50:
        ma_losses = moving_average(losses, 50)
        axes[1].plot(range(49, len(losses)), ma_losses, label='Moving avg (50 ep)', linewidth=2)
    axes[1].set_xlabel('Episode')
    axes[1].set_ylabel('Loss')
    axes[1].set_title('Training Losses')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print(f"\\n📊 Training Summary:")
    print(f"   Final 100 episodes - Avg Reward: {np.mean(rewards[-100:]):.2f}, Max: {np.max(rewards[-100:]):.2f}")
    print(f"   Best episode reward: {np.max(rewards):.2f}")

def save_model(agent, filepath):
    """Save model weights (agent's primary Q-Network: policy_net)."""
    torch.save(agent.policy_net.state_dict(), filepath)
    print(f"✅ Model saved to {filepath}")

def load_model(agent, filepath):
    """Load model weights into agent's primary Q-Network: policy_net."""
    state_dict = torch.load(filepath, map_location=agent.device)
    agent.policy_net.load_state_dict(state_dict)
    agent.policy_net.eval() # Set network to evaluation mode after loading
    print(f"✅ Model loaded from {filepath}")
    return agent

def record_video(env, agent, filename):
    """Record agent gameplay video."""
    # Set a seed for consistent visualization
    SEED = 42

    # 1. Wrap the environment for recording.
    video_env = RecordVideo(env, filename, episode_trigger=lambda x: True, name_prefix='eval_run')

    # 2. Run an episode
    state, _ = video_env.reset(seed=SEED)
    done = False
    total_reward = 0
    step = 0

    while not done:
        # *** FIX: Using agent.select_action and epsilon=0.0 ***
        # Action selection uses the trained agent's greedy policy
        action = agent.select_action(state, epsilon=0.0)

        next_state, reward, terminated, truncated, _ = video_env.step(action)
        done = terminated or truncated
        state = next_state
        total_reward += reward
        step += 1

    video_env.close()
    print(f"✅ Video recorded in the '{filename}' directory. Total Reward: {total_reward:.2f}, Steps: {step}")


In [ ]:
# ═══════════════════════════════════════════════════════════
# SECTION 7: TRAINING LOOP
# ═══════════════════════════════════════════════════════════

def train_dqn(agent, env, num_episodes):
    """
    Main training loop.

    Args:
        agent: DQNAgent instance
        env: Gymnasium environment
        num_episodes: Number of episodes to train

    Returns:
        rewards: List of episode rewards
        losses: List of training losses (avg per episode)
    """
    rewards = []
    losses = []

    epsilon = EPSILON_START
    global_step = 0

    for episode in range(num_episodes):
        state, _ = env.reset()
        done = False
        episode_reward = 0.0
        episode_losses = []

        while not done:
            # --- Epsilon-greedy action selection ---
            action = agent.select_action(state, epsilon)

            # --- Interact with environment ---
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

            # --- Store experience in replay buffer ---
            agent.replay_buffer.push(state, action, reward, next_state, done)

            state = next_state
            episode_reward += reward

            # --- Train on a random batch from replay buffer ---
            loss = agent.train_step()
            if loss is not None:
                episode_losses.append(loss)

            global_step += 1

            # --- Periodically update target network ---
            if global_step % TARGET_UPDATE == 0:
                agent.update_target_network()

        # end of episode
        rewards.append(episode_reward)
        
        # FIXED: Decay epsilon ONCE PER EPISODE (was inside while loop - too fast!)
        # This ensures proper exploration throughout training
        epsilon = max(EPSILON_END, epsilon * EPSILON_DECAY)
        if episode_losses:
            avg_loss = np.mean(episode_losses)
        else:
            avg_loss = 0.0
        losses.append(avg_loss)

        print(
            f"Episode {episode + 1}/{num_episodes} | "
            f"Reward: {episode_reward:.2f} | "
            f"Avg loss: {avg_loss:.4f} | "
            f"Epsilon: {epsilon:.3f}"
        )

    return rewards, losses

#test
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = gym.make("FlappyBird-v0", use_lidar=True)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

agent = DQNAgent(state_dim, action_dim, device=device)

rewards, losses = train_dqn(agent, env, NUM_EPISODES)

save_model(agent, MODEL_PATH)

# NEW: save metrics (optional, but nice)
np.save("rewards_v1.npy", rewards)
np.save("losses_v1.npy", losses)

# NEW: show learning curves
plot_training_results(rewards, losses)

env.close()

In [ ]:
# ═══════════════════════════════════════════════════════════
# SECTION 9.5: VIDEO
# ═══════════════════════════════════════════════════════════

LIDAR_OBS = True
ENV_ID = 'FlappyBird-v0'

MODEL_PATH = 'flappy_bird_dqn_v1.pth' # Must match the save path
VIDEO_FOLDER = './dqn_v1_performance_video'

# 1. Instantiate a new evaluation agent
eval_agent = DQNAgent(state_dim, action_dim, device=device)

# 2. Load the model
eval_agent = load_model(eval_agent, MODEL_PATH)

# 3. Create an evaluation environment (must be 'rgb_array' for video)
eval_env = gym.make(
    ENV_ID,
    render_mode='rgb_array',
    use_lidar=LIDAR_OBS
)

# 4. Record the video
record_video(eval_env, eval_agent, VIDEO_FOLDER)

In [ ]:
# ═══════════════════════════════════════════════════════════
# SECTION 9: EVALUATION
# ═══════════════════════════════════════════════════════════

def evaluate_agent(agent, env, num_episodes=10):
    """
    Evaluate agent performance without exploration (epsilon=0).

    Args:
        agent: trained DQNAgent
        env: evaluation environment (no video wrapper)
        num_episodes: number of episodes to evaluate

    Returns:
        avg_reward: Average episode reward
        avg_steps: Average number of steps survived
        scores: List of individual episode rewards
    """
    scores = []
    steps_list = []

    for ep in range(num_episodes):
        state, _ = env.reset()
        done = False
        episode_reward = 0
        ep_steps = 0

        while not done:
            # Greedy action selection (no exploration)
            action = agent.select_action(state, epsilon=0.0)

            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

            state = next_state
            episode_reward += reward
            ep_steps += 1

        scores.append(episode_reward)
        steps_list.append(ep_steps)

    avg_reward = float(np.mean(scores))
    avg_steps = float(np.mean(steps_list))

    return avg_reward, avg_steps, scores


# --- RUN EVALUATION ---
# Make a fresh evaluation environment
eval_env2 = gym.make("FlappyBird-v0", use_lidar=True)

# Evaluate the trained model (eval_agent must be loaded in Section 9.5)
avg_reward, avg_steps, scores = evaluate_agent(eval_agent, eval_env2, num_episodes=20)

eval_env2.close()

print("════════ Evaluation Results ════════")
print("Avg reward:", avg_reward)
print("Avg steps:", avg_steps)
print("Scores per episode:", scores)
print("════════════════════════════════════")

np.save("eval_scores_v1.npy", scores)
with open("eval_summary_v1.txt", "w") as f:
    f.write(f"Avg reward: {avg_reward}\n")
    f.write(f"Avg steps: {avg_steps}\n")

In [ ]:
# ═══════════════════════════════════════════════════════════
# SECTION 10: EXPERIMENTS - VERSION 1 (BASIC DQN)
# ═══════════════════════════════════════════════════════════

def run_experiment_v1(num_episodes=NUM_EPISODES):
    print("════ Running Experiment V1: Basic DQN ════")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 1) Environment + agent
    env = gym.make("FlappyBird-v0", use_lidar=True)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n

    agent = DQNAgent(state_dim, action_dim, device=device)

    # 2) Train
    rewards, losses = train_dqn(agent, env, num_episodes)
    env.close()

    # 3) Save model + metrics
    MODEL_PATH_V1 = "flappy_bird_dqn_v1.pth"
    save_model(agent, MODEL_PATH_V1)

    np.save("rewards_v1.npy", rewards)
    np.save("losses_v1.npy", losses)

    # 4) Plot learning curves
    plot_training_results(rewards, losses)

    # 5) Evaluation
    eval_env = gym.make("FlappyBird-v0", use_lidar=True)
    avg_reward, avg_steps, scores = evaluate_agent(agent, eval_env, num_episodes=20)
    eval_env.close()

    np.save("eval_scores_v1.npy", scores)
    with open("eval_summary_v1.txt", "w") as f:
        f.write(f"Avg reward: {avg_reward}\n")
        f.write(f"Avg steps: {avg_steps}\n")

    print("════ V1 Evaluation ════")
    print("Avg reward:", avg_reward)
    print("Avg steps:", avg_steps)
    print("Scores:", scores)

    # 6) Record video
    eval_env_video = gym.make("FlappyBird-v0", render_mode="rgb_array", use_lidar=True)
    VIDEO_FOLDER_V1 = "./dqn_v1_video"
    record_video(eval_env_video, agent, VIDEO_FOLDER_V1)
    eval_env_video.close()

    print("✅ Experiment V1 completed.")
    return agent, rewards, losses

# To actually run:
agent_v1, rewards_v1, losses_v1 = run_experiment_v1()


In [ ]:
# ═══════════════════════════════════════════════════════════
# SECTION 11: EXPERIMENTS - VERSION 2 (IMPROVEMENTS: DOUBLE DQN)
# ═══════════════════════════════════════════════════════════

class DoubleDQNAgent(DQNAgent):
    """
    Extends the basic DQNAgent by implementing the Double DQN update rule.
    Double DQN reduces overestimation bias and improves stability.
    """

    def train_step(self):
        # Don’t train until we have enough samples
        if len(self.replay_buffer) < max(self.batch_size, self.min_buffer_size):
            return None

        # Sample from replay buffer
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.batch_size)

        states      = torch.tensor(states, dtype=torch.float32, device=self.device)
        actions     = torch.tensor(actions, dtype=torch.long,   device=self.device).unsqueeze(1)
        rewards     = torch.tensor(rewards, dtype=torch.float32, device=self.device).unsqueeze(1)
        next_states = torch.tensor(next_states, dtype=torch.float32, device=self.device)
        dones       = torch.tensor(dones, dtype=torch.float32,  device=self.device).unsqueeze(1)

        # Q(s,a) for current states
        q_values = self.policy_net(states).gather(1, actions)

        # ---------- DOUBLE DQN TARGET UPDATE ----------
        with torch.no_grad():
            # 1) Use policy net to select best next action
            next_actions = self.policy_net(next_states).argmax(dim=1, keepdim=True)

            # 2) Use target net to evaluate that action
            next_q_target = self.target_net(next_states).gather(1, next_actions)

            target_q_values = rewards + self.gamma * (1.0 - dones) * next_q_target

        # Compute loss
        loss = self.loss_fn(q_values, target_q_values)

        self.optimizer.zero_grad()
        loss.backward()
        # FIXED: Enable gradient clipping for training stability
        torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), 1.0)
        self.optimizer.step()

        return loss.item()



def run_experiment_v2(num_episodes=NUM_EPISODES):
    """
    Run Experiment Version 2:
    - Switch from standard DQN to Double DQN
    - Train, evaluate, save metrics, and record video
    """
    print("════ Running Experiment V2: Double DQN ════")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize environment
    env = gym.make("FlappyBird-v0", use_lidar=True)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n

    # Initialize improved agent
    agent = DoubleDQNAgent(state_dim, action_dim, device=device)

    # Train Double DQN agent
    rewards, losses = train_dqn(agent, env, num_episodes)
    env.close()

    # Save model & training metrics
    MODEL_PATH_V2 = "flappy_bird_dqn_v2_double.pth"
    save_model(agent, MODEL_PATH_V2)

    np.save("rewards_v2.npy", rewards)
    np.save("losses_v2.npy", losses)

    # Plot learning curves
    plot_training_results(rewards, losses)

    # Evaluate on fresh environment
    eval_env = gym.make("FlappyBird-v0", use_lidar=True)
    avg_reward, avg_steps, scores = evaluate_agent(agent, eval_env, num_episodes=20)
    eval_env.close()

    # Save evaluation results
    np.save("eval_scores_v2.npy", scores)
    with open("eval_summary_v2.txt", "w") as f:
        f.write(f"Avg reward: {avg_reward}\n")
        f.write(f"Avg steps: {avg_steps}\n")

    print("════ V2 Evaluation Results ════")
    print("Avg reward:", avg_reward)
    print("Avg steps:", avg_steps)
    print("Scores:", scores)

    # Record video of trained agent
    eval_env_video = gym.make("FlappyBird-v0", render_mode="rgb_array", use_lidar=True)
    VIDEO_FOLDER_V2 = "./dqn_v2_double_video"
    record_video(eval_env_video, agent, VIDEO_FOLDER_V2)
    eval_env_video.close()

    print("✅ Experiment V2 completed.")
    return agent, rewards, losses

# To run this experiment:
agent_v2, rewards_v2, losses_v2 = run_experiment_v2()


In [ ]:
# ═══════════════════════════════════════════════════════════
# SECTION 12: EXPERIMENTS - VERSION 3 (MORE IMPROVEMENTS)
# ═══════════════════════════════════════════════════════════

def train_dqn_with_reward_shaping(agent, env, num_episodes,
                                  step_reward=0.1,
                                  epsilon_start=1.0,
                                  epsilon_end=0.01,
                                  epsilon_decay=0.997):
    """
    Training loop with reward shaping and tuned epsilon decay.
    """
    rewards = []
    losses = []

    epsilon = epsilon_start
    global_step = 0

    for episode in range(num_episodes):
        state, _ = env.reset()
        done = False
        episode_reward = 0.0
        episode_losses = []

        while not done:
            action = agent.select_action(state, epsilon)

            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

            # ---------- REWARD SHAPING ----------
            shaped_reward = reward + step_reward  # small bonus for staying alive

            agent.replay_buffer.push(state, action, shaped_reward, next_state, done)

            state = next_state
            episode_reward += shaped_reward

            loss = agent.train_step()
            if loss is not None:
                episode_losses.append(loss)

            global_step += 1
            if global_step % TARGET_UPDATE == 0:
                agent.update_target_network()

        rewards.append(episode_reward)
        
        # FIXED: Decay epsilon ONCE PER EPISODE (was inside while loop)
        epsilon = max(epsilon_end, epsilon * epsilon_decay)
        losses.append(np.mean(episode_losses) if episode_losses else 0.0)

        print(
            f"[V3] Episode {episode+1}/{num_episodes} "
            f"Reward: {episode_reward:.2f} "
            f"Avg loss: {losses[-1]:.4f} "
            f"Epsilon: {epsilon:.3f}"
        )

    return rewards, losses


def run_experiment_v3(num_episodes=NUM_EPISODES):
    print("════ Running Experiment V3: Double DQN + Reward Shaping ════")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    env = gym.make("FlappyBird-v0", use_lidar=True)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n

    # Use DoubleDQNAgent again (stacking improvements)
    agent = DoubleDQNAgent(state_dim, action_dim, device=device)

    rewards, losses = train_dqn_with_reward_shaping(
        agent, env, num_episodes,
        step_reward=0.01,
        epsilon_start=1.0,
        epsilon_end=0.05,
        epsilon_decay=0.997
    )
    env.close()

    MODEL_PATH_V3 = "flappy_bird_dqn_v3_shaped.pth"
    save_model(agent, MODEL_PATH_V3)

    np.save("rewards_v3.npy", rewards)
    np.save("losses_v3.npy", losses)

    plot_training_results(rewards, losses)

    eval_env = gym.make("FlappyBird-v0", use_lidar=True)
    avg_reward, avg_steps, scores = evaluate_agent(agent, eval_env, num_episodes=20)
    eval_env.close()

    np.save("eval_scores_v3.npy", scores)
    with open("eval_summary_v3.txt", "w") as f:
        f.write(f"Avg reward: {avg_reward}\n")
        f.write(f"Avg steps: {avg_steps}\n")

    print("════ V3 Evaluation ════")
    print("Avg reward:", avg_reward)
    print("Avg steps:", avg_steps)

    eval_env_video = gym.make("FlappyBird-v0", render_mode="rgb_array", use_lidar=True)
    VIDEO_FOLDER_V3 = "./dqn_v3_shaped_video"
    record_video(eval_env_video, agent, VIDEO_FOLDER_V3)
    eval_env_video.close()

    print("✅ Experiment V3 completed.")
    return agent, rewards, losses

# Run:
agent_v3, rewards_v3, losses_v3 = run_experiment_v3()

In [ ]:
# ═══════════════════════════════════════════════════════════
# SECTION 13: RESULTS & ANALYSIS
# ═══════════════════════════════════════════════════════════

def compare_learning_curves():
    """Plot rewards of V1, V2, V3 on one graph with moving averages."""
    rewards_v1 = np.load("rewards_v1.npy")
    rewards_v2 = np.load("rewards_v2.npy")
    rewards_v3 = np.load("rewards_v3.npy")

    fig, axes = plt.subplots(1, 2, figsize=(16, 5))
    
    # Raw rewards (transparent)
    axes[0].plot(rewards_v1, alpha=0.2, color='blue')
    axes[0].plot(rewards_v2, alpha=0.2, color='orange')
    axes[0].plot(rewards_v3, alpha=0.2, color='green')
    
    # Moving averages (solid lines)
    window = 50
    if len(rewards_v1) >= window:
        ma_v1 = moving_average(rewards_v1, window)
        axes[0].plot(range(window-1, len(rewards_v1)), ma_v1, label="V1: Basic DQN", linewidth=2, color='blue')
    if len(rewards_v2) >= window:
        ma_v2 = moving_average(rewards_v2, window)
        axes[0].plot(range(window-1, len(rewards_v2)), ma_v2, label="V2: Double DQN", linewidth=2, color='orange')
    if len(rewards_v3) >= window:
        ma_v3 = moving_average(rewards_v3, window)
        axes[0].plot(range(window-1, len(rewards_v3)), ma_v3, label="V3: Double DQN + Shaping", linewidth=2, color='green')
    
    axes[0].set_xlabel("Episode")
    axes[0].set_ylabel("Episode reward")
    axes[0].set_title("Training Reward Comparison (Moving Avg)")
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.5, label='Random baseline')
    
    # Bar chart comparison
    versions = ['V0\nRandom', 'V1\nBasic DQN', 'V2\nDouble DQN', 'V3\nDouble+Shaping']
    final_rewards = [
        -7.9,  # Random baseline
        np.mean(rewards_v1[-100:]),
        np.mean(rewards_v2[-100:]),
        np.mean(rewards_v3[-100:])
    ]
    colors = ['red', 'blue', 'orange', 'green']
    bars = axes[1].bar(versions, final_rewards, color=colors, alpha=0.7)
    axes[1].set_ylabel("Average Reward (last 100 episodes)")
    axes[1].set_title("Final Performance Comparison")
    axes[1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    axes[1].grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bar, val in zip(bars, final_rewards):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                    f'{val:.1f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.show()


def print_eval_summaries():
    """Print textual comparison of evaluation results."""
    def load_summary(path):
        with open(path, "r") as f:
            lines = f.readlines()
        avg_reward = float(lines[0].split(":")[1])
        avg_steps = float(lines[1].split(":")[1])
        return avg_reward, avg_steps

    avg_r1, avg_s1 = load_summary("eval_summary_v1.txt")
    avg_r2, avg_s2 = load_summary("eval_summary_v2.txt")
    avg_r3, avg_s3 = load_summary("eval_summary_v3.txt")

    print("════ FINAL COMPARISON ════")
    print(f"V1 Basic DQN:       reward={avg_r1:.2f}, steps={avg_s1:.1f}")
    print(f"V2 Double DQN:      reward={avg_r2:.2f}, steps={avg_s2:.1f}")
    print(f"V3 Double+Shaping:  reward={avg_r3:.2f}, steps={avg_s3:.1f}")

# Side-by-side videos are already produced as different folders:
#random_agent_video/, dqn_v1_video/, dqn_v2_double_video/, dqn_v3_shaped_video/

# After running all experiments:
compare_learning_curves()
print_eval_summaries()